In [1]:
import tensorflow as tf
from tensorflow import keras

C:\Users\BastOS\anaconda3\envs\timeseries\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\BastOS\anaconda3\envs\timeseries\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\BastOS\anaconda3\envs\timeseries\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\BastOS\anaconda3\envs\timeseries\lib\site-pack

# Univariate LSTM

In [2]:
import numpy as np

In [3]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    
    for i in range(len(sequence)):
        end_ix = i + n_steps

        # De 0 à 8
        if end_ix > len(sequence) -1:
            break
        
        seq_x , seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [4]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

n_steps = 3

X, y = split_sequence(raw_seq, n_steps)

print(X.shape, y.shape)

for i in range(len(X)):
    print(X[i], y[i])

(6, 3) (6,)
[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [5]:
# How many time series in parallel?
n_features = 1

## Vanilla LSTM

In [6]:
# define model
model = keras.models.Sequential([
    keras.layers.LSTM(50, activation="relu", input_shape=(n_steps, n_features)),
    keras.layers.Dense(1)  
])

model.compile(optimizer="adam", loss="mse")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
# reshape data into 3 dim
X = X.reshape(X.shape[0], X.shape[1], n_features)

In [8]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
# demonstrate prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[102.2485]]


## Stacked LSTM

In [10]:
# define model
model = keras.models.Sequential([
    keras.layers.LSTM(50, activation="relu", return_sequences=True, input_shape=(n_steps, n_features)),
    keras.layers.LSTM(50, activation="relu"),
    keras.layers.Dense(1)  
])

model.compile(optimizer="adam", loss="mse")

In [11]:
# fit stacked lstm
model.fit(X, y, epochs=200, verbose=0)

In [12]:
# demonstrate prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[102.94471]]


## Bidirectional LSTM

In [13]:
# define model
model = keras.models.Sequential([
    keras.layers.Bidirectional(keras.layers.LSTM(50, activation="relu"), input_shape=(n_steps, n_features)),
    keras.layers.Dense(1)  
])

model.compile(optimizer="adam", loss="mse")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
# fit bidirectional lstm
model.fit(X, y, epochs=200, verbose=0)

In [15]:
# demonstrate prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[101.57055]]


## CNN-LSTM

In [16]:
# choose a number of stime steps
n_steps = 4

# split into samples
X, y = split_sequence(raw_seq, n_steps)

# reshape from [samples, timesteps] into [samples, subsequences, subtimesteps, features]
n_features = 1
n_seq = 2
n_substeps = 2

X = X.reshape(X.shape[0], n_seq, n_substeps, n_features)

In [17]:
# define the input cnn model
# This can be achieved by wrapping the entire CNN model in a TimeDistributed wrapper that
# will apply the entire model once per input, in this case, once per input subsequence.

model = keras.models.Sequential()
model.add(keras.layers.TimeDistributed(keras.layers.Conv1D(filters=64, kernel_size=1, activation='relu'),
                                       input_shape=(None, n_substeps, n_features)))
model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling1D(pool_size=2)))   
model.add(keras.layers.TimeDistributed(keras.layers.Flatten()))   

In [18]:
# define the output model
model.add(keras.layers.LSTM(50, activation="relu"))
model.add(keras.layers.Dense(1))

model.compile(optimizer="adam", loss="mse")

In [19]:
# fit hybrid cnn-lstm
model.fit(X, y, epochs=500, verbose=0)

In [20]:
# demonstrate prediction
x_input = np.array([60, 70, 80, 90])
x_input = x_input.reshape(1, n_seq, n_substeps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[107.40256]]


## ConvLSTM

In [21]:
# choose a number of stime steps
n_steps = 4

# split into samples
X, y = split_sequence(raw_seq, n_steps)

# reshape from [samples, timesteps] into [samples, timesteps, rows, columns, features]
n_features = 1
n_seq = 2
n_substeps = 2

X = X.reshape(X.shape[0], n_seq, 1, n_substeps, n_features)

In [22]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import ConvLSTM2D

model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_substeps, n_features)))
model.add(Flatten())
model.add(Dense(1))

model.compile(optimizer="adam", loss="mse")

Using TensorFlow backend.


In [23]:
model.fit(X, y, epochs=500, verbose=0)

In [24]:
# demonstrate prediction
x_input = np.array([60, 70, 80, 90])
x_input = x_input.reshape(1, n_seq, 1, n_substeps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[103.86982]]


# Multivariate LSTM

## Multiple Input Series

In [25]:
# define input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i] + in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, cols] structure
in_seq1 = in_seq1.reshape(in_seq1.shape[0], 1)
in_seq2 = in_seq2.reshape(in_seq2.shape[0], 1)
out_seq = out_seq.reshape(out_seq.shape[0], 1)

# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [26]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps
        
        if end_ix > len(sequences):
            break
        
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [27]:
n_steps = 3

X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

for i in range(len(X)):
    print(X[i], y[i])

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [28]:
n_features = X.shape[2]

In [29]:
from keras.layers import LSTM

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

In [30]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [31]:
# demonstrate prediction
x_input = np.array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[206.2151]]


## Multiple Parallel Series

In [32]:
# define input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i] + in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, cols] structure
in_seq1 = in_seq1.reshape(in_seq1.shape[0], 1)
in_seq2 = in_seq2.reshape(in_seq2.shape[0], 1)
out_seq = out_seq.reshape(out_seq.shape[0], 1)

# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [33]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps
        
        if end_ix > len(sequences)-1:
            break
        
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    
    return np.array(X), np.array(y)

In [34]:
n_steps = 3

X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

for i in range(len(X)):
    print(X[i], y[i])

(6, 3, 3) (6, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


In [35]:
n_features = X.shape[2]

In [36]:
# define model
model = Sequential()
model.add(LSTM(100, activation="relu", return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation="relu"))
model.add(Dense(n_features))

model.compile(optimizer='adam', loss='mse')

In [37]:
# fit model
model.fit(X, y, epochs=400, verbose=0)

In [38]:
# demonstrate prediction
x_input = np.array([[70, 75, 145], [80, 85, 165], [90, 95, 185]])
print(x_input.shape)
x_input = x_input.reshape(1, n_steps, n_features)
print(x_input.shape)
yhat = model.predict(x_input)
print(yhat)

(3, 3)
(1, 3, 3)
[[100.3972   105.633896 205.67896 ]]


# Multi-step LSTM

In [39]:
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    
    for i in range(len(sequence)):
        end_ix = i + n_steps_in
        out_ix = end_ix + n_steps_out
        
        if out_ix > len(sequence):
            break
        
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [40]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

n_steps_in, n_steps_out = 3, 2

X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

print(X.shape, y.shape)

for i in range(len(X)):
    print(X[i], y[i])

(5, 3) (5, 2)
[10 20 30] [40 50]
[20 30 40] [50 60]
[30 40 50] [60 70]
[40 50 60] [70 80]
[50 60 70] [80 90]


## Vector Output Model

In [41]:
# output a vector directly that can be interpreted as a multi-step forecast (seen previously)

n_features = 1

# reshape into 3 dim
X = X.reshape(X.shape[0], X.shape[1], n_features)

In [42]:
# define model
model = Sequential()
model.add(LSTM(100, activation="relu", return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation="relu"))
model.add(Dense(n_steps_out))

model.compile(optimizer='adam', loss='mse')

In [43]:
# fit model
model.fit(X, y, epochs=400, verbose=0)

In [44]:
# demonstrate prediction
x_input = np.array([60, 70, 80])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[ 91.411446 102.52533 ]]


## Encoder-Decoder Model

In [45]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

n_steps_in, n_steps_out = 3, 2

X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

print(X.shape, y.shape)

(5, 3) (5, 2)


In [46]:
n_features = 1

X = X.reshape(X.shape[0], X.shape[1], n_features)

In [47]:
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

# define encoder model
# output of the encoder is a fixed length vector that represents the model's interpretation of the sequence
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features)))

# The decoder uses the output of the encoder as an input
# First, the fixed-length output of the encoder is repeated, once for each requirend time step in the output sequence
# repeat encoding
model.add(RepeatVector(n_steps_out))

# define decoder model
model.add(LSTM(100, activation='relu', return_sequences=True))

# define model output
model.add(TimeDistributed(Dense(1)))

model.compile(optimizer='adam', loss='mse')

In [48]:
# reshape output
y = y.reshape(y.shape[0], y.shape[1], n_features)

In [49]:
# fit model
model.fit(X, y, epochs=100, verbose=0)

In [50]:
# demonstrate prediction
x_input = np.array([60, 70, 80])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[[ 92.49516 ]
  [102.108154]]]


# Multivariate Multi-step LSTM Models

## Multiple Input Multi-step Output

In [51]:
# define input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i] + in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, cols] structure
in_seq1 = in_seq1.reshape(in_seq1.shape[0], 1)
in_seq2 = in_seq2.reshape(in_seq2.shape[0], 1)
out_seq = out_seq.reshape(out_seq.shape[0], 1)

# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [52]:
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        out_ix = end_ix + n_steps_out -1
        
        if out_ix > len(sequences):
            break
        
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [53]:
n_steps_in, n_steps_out = 3, 2

X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)

for i in range(len(X)):
    print(X[i], y[i])

(6, 3, 2) (6, 2)
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]


In [54]:
n_features = X.shape[2]

In [55]:
# define model
model = Sequential()
model.add(LSTM(100, activation="relu", return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation="relu"))
model.add(Dense(n_steps_out))

model.compile(optimizer='adam', loss='mse')

In [56]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [57]:
# demonstrate prediction
x_input = np.array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[187.24307 208.91602]]


## Multiple Parallel Input Multi-step Output

In [58]:
# define input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i] + in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, cols] structure
in_seq1 = in_seq1.reshape(in_seq1.shape[0], 1)
in_seq2 = in_seq2.reshape(in_seq2.shape[0], 1)
out_seq = out_seq.reshape(out_seq.shape[0], 1)

# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [59]:
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        out_ix = end_ix + n_steps_out
        
        if out_ix > len(sequences):
            break
        
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_ix, :]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [67]:
n_steps_in, n_steps_out = 3, 2

X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)

for i in range(len(X)):
    print(X[i], y[i])

(5, 3, 3) (5, 2, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [[ 50  55 105]
 [ 60  65 125]]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [[ 60  65 125]
 [ 70  75 145]]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [[ 70  75 145]
 [ 80  85 165]]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [[ 80  85 165]
 [ 90  95 185]]


In [61]:
# We can use Vector Output or Encoder-Decoder LSTM to model this problem
n_features = X.shape[2]
n_output = y.shape[1] * y.shape[2]

y = y.reshape(y.shape[0], n_output)

In [62]:
# define model Output Vector LSTM
model = Sequential()
model.add(LSTM(100, activation="relu", return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation="relu"))
model.add(Dense(n_output))

model.compile(optimizer='adam', loss='mse')

In [63]:
# fit the model
model.fit(X, y, epochs=200, verbose=0)

In [64]:
# demonstrate prediction
x_input = np.array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[ 89.76642  95.25615 186.46371 100.38029 107.02497 208.14716]]


In [65]:
# define model Encoder-Decoder LSTM
model = Sequential()
model.add(LSTM(100, activation="relu", input_shape=(n_steps_in, n_features)))

model.add(RepeatVector(n_steps_out))

model.add(LSTM(100, return_sequences=True, activation="relu"))

model.add(TimeDistributed(Dense(n_features)))

model.compile(optimizer='adam', loss='mse')

In [68]:
model.fit(X, y, epochs=300, verbose=0)

In [69]:
# demonstrate prediction
x_input = np.array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[[ 91.501945  95.53477  185.0618  ]
  [101.03532  104.86024  206.37529 ]]]
